# Analyze Cancer Observations with Spark Machine Learning
The data in use is from the <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)"><b>Wisconsin Diagnostic Breast Cancer (WDBC) Data Set</b></a> which categorizes breast tumor cases as either benign or malignant based on 9 features to predict the diagnosis. For each cancer observation, we have the following information:

1\. Sample code number: id number<br/>
2\. Clump Thickness: 1 - 10<br/>
3\. Uniformity of Cell Size: 1 - 10<br/>
4\. Uniformity of Cell Shape: 1 - 10<br/>
5\. Marginal Adhesion: 1 - 10<br/>
6\. Single Epithelial Cell Size: 1 - 10<br/>
7\. Bare Nuclei: 1 - 10<br/>
8\. Bland Chromatin: 1 - 10<br/>
9\. Normal Nucleoli: 1 - 10<br/>
10\. Mitoses: 1 - 10<br/>
11\. Class: (2 for benign, 4 for malignant)<br/>

The Cancer Observation data file has the following format:<br/>
1000025,5,1,1,1,2,1,3,1,1,2<br/>
1002945,5,4,4,5,7,10,3,2,1,2<br/>
1015425,3,1,1,1,2,2,3,1,1,2

In this scenario, we will build a logistic regression model to predict the label / classification of malignant or not based on the following features:
<ul>
    <li>Label → malignant or benign (1 or 0)</li>
    <li>Features → {Clump Thickness, Uniformity of Cell Size, Uniformity of Cell Shape, Marginal Adhesion, Single Epithelial Cell Size, Bare Nuclei, Bland Chromatin, Normal Nucleoli, Mitoses }</li>
</ul>

Spark ML used in this example
<img src="img/bcmlprocess.png">

# Load and Parse the Data
Import the machine learning packages.

In [1]:
import org.apache.spark._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SQLContext
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler

val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._
import sqlContext._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.linalg.DenseVector
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

sqlContext = org.apache.spark.sql.SQLContext@688ed34


org.apache.spark.sql.SQLContext@688ed34

Use a Scala case class to define the schema corresponding to a line in the data file.

In [7]:
/** define the Cancer Observation Schema */
case class Obs(
    clas: Double,
    thickness: Double,
    size: Double,
    shape: Double,
    madh: Double,
    epsize: Double,
    bnuc: Double,
    bchrom: Double,
    nNuc: Double,
    mit: Double
)

defined class Obs


The functions below parse a line from the data file into the Cancer Observation class.

In [17]:
/* 
 * function to create a Obs class from an Array of Double.
 * Class Malignant 4 is changed to 1
 */
def parseObs(line: Array[Double]): Obs = {
    Obs(
        if (line(9) == 4.0) 1 else 0, line(0), line(1), line(2), line(3), line(4), line(5), line(6), line(7), line(8)
    )
}

/* 
 * function to transform an RDD of Strings into an RDD of Double,
 * filter lines with ?, remove first column
 */
def parseRDD(rdd: RDD[String]): RDD[Array[Double]] = {
    rdd.map(_.split(","))
       .filter(_(6) != "?")
       .map(_.drop(1))
       .map(_.map(_.toDouble))
}

parseObs: (line: Array[Double])Obs
parseRDD: (rdd: org.apache.spark.rdd.RDD[String])org.apache.spark.rdd.RDD[Array[Double]]


Below we load the data from the csv file into an RDD of Strings. Then we use the map transformation on the rdd, which will apply the ParseRDD function to transform each String element in the RDD into an Array of Double. Then we use another map transformation, which will apply the ParseObs function to transform each Array of Double in the RDD into an Array of Cancer Observation objects. The toDF() method transforms the RDD of Array[[Cancer Observation]] into a Dataframe with the Cancer Observation class schema.
<img src="img/bcloaddata.png">

In [18]:
// load the data into a DataFrame
val rdd = sc.textFile("data/breast-cancer-wisconsin.data")
val obsRDD = parseRDD(rdd).map(parseObs)
val obsDF = obsRDD.toDF().cache()
obsDF.registerTempTable("obs")

rdd = data/breast-cancer-wisconsin.data MapPartitionsRDD[25] at textFile at <console>:62
obsRDD = MapPartitionsRDD[30] at map at <console>:63
obsDF = [clas: double, thickness: double ... 8 more fields]


[clas: double, thickness: double ... 8 more fields]

DataFrame printSchema() Prints the schema to the console in a tree format

In [19]:
// Return the schema of this DataFrame
obsDF.printSchema

root
 |-- clas: double (nullable = false)
 |-- thickness: double (nullable = false)
 |-- size: double (nullable = false)
 |-- shape: double (nullable = false)
 |-- madh: double (nullable = false)
 |-- epsize: double (nullable = false)
 |-- bnuc: double (nullable = false)
 |-- bchrom: double (nullable = false)
 |-- nNuc: double (nullable = false)
 |-- mit: double (nullable = false)



In [20]:
// Display the top 20 rows of DataFrame
obsDF.show

+----+---------+----+-----+----+------+----+------+----+---+
|clas|thickness|size|shape|madh|epsize|bnuc|bchrom|nNuc|mit|
+----+---------+----+-----+----+------+----+------+----+---+
| 0.0|      5.0| 1.0|  1.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|
| 0.0|      5.0| 4.0|  4.0| 5.0|   7.0|10.0|   3.0| 2.0|1.0|
| 0.0|      3.0| 1.0|  1.0| 1.0|   2.0| 2.0|   3.0| 1.0|1.0|
| 0.0|      6.0| 8.0|  8.0| 1.0|   3.0| 4.0|   3.0| 7.0|1.0|
| 0.0|      4.0| 1.0|  1.0| 3.0|   2.0| 1.0|   3.0| 1.0|1.0|
| 1.0|      8.0|10.0| 10.0| 8.0|   7.0|10.0|   9.0| 7.0|1.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   2.0|10.0|   3.0| 1.0|1.0|
| 0.0|      2.0| 1.0|  2.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|
| 0.0|      2.0| 1.0|  1.0| 1.0|   2.0| 1.0|   1.0| 1.0|5.0|
| 0.0|      4.0| 2.0|  1.0| 1.0|   2.0| 1.0|   2.0| 1.0|1.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   1.0| 1.0|   3.0| 1.0|1.0|
| 0.0|      2.0| 1.0|  1.0| 1.0|   2.0| 1.0|   2.0| 1.0|1.0|
| 1.0|      5.0| 3.0|  3.0| 3.0|   2.0| 3.0|   4.0| 4.0|1.0|
| 0.0|      1.0| 1.0|  1

Query dataframe using SQL queries. The funtions are provided by the Scala DataFrame API.

In [25]:
//  describe computes statistics for thickness column, including count, mean, stddev, min, and max
obsDF.describe("thickness").show

+-------+------------------+
|summary|         thickness|
+-------+------------------+
|  count|               683|
|   mean|  4.44216691068814|
| stddev|2.8207613188371288|
|    min|               1.0|
|    max|              10.0|
+-------+------------------+



In [29]:
// compute the avg thickness, size, shape grouped by clas (malignant or not)
sqlContext.sql("SELECT clas, avg(thickness) as avgthickness, avg(size) as avgsize, avg(shape) as avgshape FROM obs GROUP BY clas").show

+----+-----------------+------------------+------------------+
|clas|     avgthickness|           avgsize|          avgshape|
+----+-----------------+------------------+------------------+
| 0.0|2.963963963963964|1.3063063063063063|1.4144144144144144|
| 1.0|7.188284518828452| 6.577405857740586| 6.560669456066946|
+----+-----------------+------------------+------------------+



In [30]:
// compute avg thickness grouped by clas (malignant or not)
obsDF.groupBy("clas").avg("thickness").show

+----+-----------------+
|clas|   avg(thickness)|
+----+-----------------+
| 0.0|2.963963963963964|
| 1.0|7.188284518828452|
+----+-----------------+



# Extract Features
To build a classifier model, you first extract the features that most contribute to the classification. In the cancer data set, the data is labeled with two classes – 1 (malignant) and 0 (not malignant).

The features for each item consists of the fields shown below:
<ul>
    <li>Label → malignant: 0 or 1</li>
    <li>Features → {"thickness", "size", "shape", "madh", "epsize", "bnuc", "bchrom", "nNuc", "mit"}</li>
</ul>

# Define Features Array
In order for the features to be used by a machine learning algorithm, the features are transformed and put into Feature Vectors, which are vectors of numbers representing the value for each feature.

Below a VectorAssembler is used to transform and return a new DataFrame with all of the feature columns in a vector column
<img src="img/bctransformfeatures.png">

In [34]:
//define the feature columns to put in the feature vector
val featureCols = Array("thickness", "size", "shape", "madh", "epsize", "bnuc", "bchrom", "nNuc", "mit")

//set the input and output column names
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")

//return a dataframe with all of the  feature columns in  a vector column
val df2 = assembler.transform(obsDF)

// the transform method produced a new column: features.
df2.show(10)

+----+---------+----+-----+----+------+----+------+----+---+--------------------+
|clas|thickness|size|shape|madh|epsize|bnuc|bchrom|nNuc|mit|            features|
+----+---------+----+-----+----+------+----+------+----+---+--------------------+
| 0.0|      5.0| 1.0|  1.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|[5.0,1.0,1.0,1.0,...|
| 0.0|      5.0| 4.0|  4.0| 5.0|   7.0|10.0|   3.0| 2.0|1.0|[5.0,4.0,4.0,5.0,...|
| 0.0|      3.0| 1.0|  1.0| 1.0|   2.0| 2.0|   3.0| 1.0|1.0|[3.0,1.0,1.0,1.0,...|
| 0.0|      6.0| 8.0|  8.0| 1.0|   3.0| 4.0|   3.0| 7.0|1.0|[6.0,8.0,8.0,1.0,...|
| 0.0|      4.0| 1.0|  1.0| 3.0|   2.0| 1.0|   3.0| 1.0|1.0|[4.0,1.0,1.0,3.0,...|
| 1.0|      8.0|10.0| 10.0| 8.0|   7.0|10.0|   9.0| 7.0|1.0|[8.0,10.0,10.0,8....|
| 0.0|      1.0| 1.0|  1.0| 1.0|   2.0|10.0|   3.0| 1.0|1.0|[1.0,1.0,1.0,1.0,...|
| 0.0|      2.0| 1.0|  2.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|[2.0,1.0,2.0,1.0,...|
| 0.0|      2.0| 1.0|  1.0| 1.0|   2.0| 1.0|   1.0| 1.0|5.0|[2.0,1.0,1.0,1.0,...|
| 0.0|      4.0|

featureCols = Array(thickness, size, shape, madh, epsize, bnuc, bchrom, nNuc, mit)
assembler = vecAssembler_333a2c001e2b
df2 = [clas: double, thickness: double ... 9 more fields]


[clas: double, thickness: double ... 9 more fields]

Next, use a StringIndexer to return a Dataframe with the clas (malignant or not) column added as a label.
<img src="img/bctransformfeaturesandlabel.png">

In [33]:
//  Create a label column with the StringIndexer
val labelIndexer = new StringIndexer().setInputCol("clas").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

// the  transform method produced a new column: label.
df3.show(10)

+----+---------+----+-----+----+------+----+------+----+---+--------------------+-----+
|clas|thickness|size|shape|madh|epsize|bnuc|bchrom|nNuc|mit|            features|label|
+----+---------+----+-----+----+------+----+------+----+---+--------------------+-----+
| 0.0|      5.0| 1.0|  1.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|[5.0,1.0,1.0,1.0,...|  0.0|
| 0.0|      5.0| 4.0|  4.0| 5.0|   7.0|10.0|   3.0| 2.0|1.0|[5.0,4.0,4.0,5.0,...|  0.0|
| 0.0|      3.0| 1.0|  1.0| 1.0|   2.0| 2.0|   3.0| 1.0|1.0|[3.0,1.0,1.0,1.0,...|  0.0|
| 0.0|      6.0| 8.0|  8.0| 1.0|   3.0| 4.0|   3.0| 7.0|1.0|[6.0,8.0,8.0,1.0,...|  0.0|
| 0.0|      4.0| 1.0|  1.0| 3.0|   2.0| 1.0|   3.0| 1.0|1.0|[4.0,1.0,1.0,3.0,...|  0.0|
| 1.0|      8.0|10.0| 10.0| 8.0|   7.0|10.0|   9.0| 7.0|1.0|[8.0,10.0,10.0,8....|  1.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   2.0|10.0|   3.0| 1.0|1.0|[1.0,1.0,1.0,1.0,...|  0.0|
| 0.0|      2.0| 1.0|  2.0| 1.0|   2.0| 1.0|   3.0| 1.0|1.0|[2.0,1.0,2.0,1.0,...|  0.0|
| 0.0|      2.0| 1.0|  1.0| 1.0|

labelIndexer = strIdx_bfc548d6e197
df3 = [clas: double, thickness: double ... 10 more fields]


[clas: double, thickness: double ... 10 more fields]

Below the data is split into a training data set and a test data set, 70% of the data is used to train the model, and 30% will be used for testing.

In [35]:
//  split the dataframe into training and test data
val splitSeed = 5043
val Array(trainingData, testData) = df3.randomSplit(Array(0.7, 0.3), splitSeed)

splitSeed = 5043
trainingData = [clas: double, thickness: double ... 10 more fields]
testData = [clas: double, thickness: double ... 10 more fields]


[clas: double, thickness: double ... 10 more fields]

# Train the Model
<img src="img/creditmlcrossvalidation.png">
Next, we train the logistic regression model with elastic net regularization

The model is trained by making associations between the input features and the labeled output associated with those features.
<img src="img/bcfitmodel.png">

In [36]:
// create the classifier,  set parameters for training
val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

// use logistic regression to train (fit) the model with the training data
val model = lr.fit(trainingData)    

// Print the coefficients and intercept for logistic regression**
println(s"Coefficients: ${model.coefficients} Intercept: ${model.intercept}")

Coefficients: [0.0,0.06503554553146344,0.07181362361391282,0.0,0.0,0.07583963853124735,0.0012675057388237378,0.0,0.0] Intercept: -1.393191423126092


lr = logreg_a135e303a107
model = logreg_a135e303a107


logreg_a135e303a107

# Test the Model
Next we use the test data to get predictions.
<img src="img/bcpredicttest.png">

In [39]:
// run the  model on test features to get predictions
val predictions = model.transform(testData)

// As you can see, the previous model transform produced a new columns: rawPrediction, probablity and prediction.**
predictions.show(10)

+----+---------+----+-----+----+------+----+------+----+---+--------------------+-----+--------------------+--------------------+----------+
|clas|thickness|size|shape|madh|epsize|bnuc|bchrom|nNuc|mit|            features|label|       rawPrediction|         probability|prediction|
+----+---------+----+-----+----+------+----+------+----+---+--------------------+-----+--------------------+--------------------+----------+
| 0.0|      1.0| 1.0|  1.0| 1.0|   1.0| 1.0|   1.0| 3.0|1.0|[1.0,1.0,1.0,1.0,...|  0.0|[1.17923510971064...|[0.76481024658406...|       0.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   1.0| 1.0|   3.0| 1.0|1.0|[1.0,1.0,1.0,1.0,...|  0.0|[1.17670009823299...|[0.76435395397908...|       0.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   1.0| 1.0|   3.0| 1.0|1.0|[1.0,1.0,1.0,1.0,...|  0.0|[1.17670009823299...|[0.76435395397908...|       0.0|
| 0.0|      1.0| 1.0|  1.0| 1.0|   2.0| 1.0|   1.0| 1.0|1.0|[1.0,1.0,1.0,1.0,...|  0.0|[1.17923510971064...|[0.76481024658406...|       0.0|
| 0.0|      1

predictions = [clas: double, thickness: double ... 13 more fields]


[clas: double, thickness: double ... 13 more fields]

Below we evaluate the predictions, we use a BinaryClassificationEvaluator which returns a precision metric by comparing the test label column with the test prediction column. In this case, the evaluation returns 99% precision.
<img src="img/bcevaluatemodelpredictions.png">
A common metric used for logistic regression is area under the ROC curve (AUC). We can use the <b>BinaryClasssificationEvaluator</b> to obtain the AUC.

In [41]:
// create an Evaluator for binary classification, which expects two input columns: rawPrediction and label.
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("rawPrediction").setMetricName("areaUnderROC")

// Evaluates predictions and returns a scalar metric areaUnderROC(larger is better).**
val accuracy = evaluator.evaluate(predictions)

evaluator = binEval_cf70e1b1178d
accuracy = 0.9926910299003322


0.9926910299003322

Below we calculate some more metrics. The number of false and true positive and negative predictions is also useful:

<ul>
    <li>True positives are how often the model correctly predicted a tumour was malignant</li>
    <li>False positives are how often the model predicted a tumour was malignant when it was benign</li>
    <li>True negatives indicate how the model correctly predicted a tumour was benign</li>
    <li>False negatives indicate how often the model predicted a tumour was benign when in fact it was malignant</li>
</ul>

In [43]:
// Calculate Metrics
val lp = predictions.select( "label", "prediction")
val counttotal = predictions.count()
val correct = lp.filter($"label" === $"prediction").count()
val wrong = lp.filter(not($"label" === $"prediction")).count()
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count()
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count()
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count()
val ratioWrong=wrong.toDouble/counttotal.toDouble
val ratioCorrect=correct.toDouble/counttotal.toDouble

lp = [label: double, prediction: double]
counttotal = 199
correct = 168
wrong = 31
truep = 128
falseN = 30
falseP = 1
ratioWrong = 0.15577889447236182
ratioCorrect = 0.8442211055276382


0.8442211055276382

In [45]:
// use MLlib to evaluate, convert DF to RDD
val predictionAndLabels = predictions.select("rawPrediction", "label").rdd.map(x => (x(0).asInstanceOf[DenseVector](1), x(1).asInstanceOf[Double]))
val metrics = new BinaryClassificationMetrics(predictionAndLabels)
println("area under the precision-recall curve: " + metrics.areaUnderPR)
println("area under the receiver operating characteristic (ROC) curve : " + metrics.areaUnderROC)

// A Precision-Recall curve plots (precision, recall) points for different threshold values, while a receiver operating characteristic, or ROC, curve plots (recall, false positive rate) points. The closer  the area Under ROC is to 1, the better the model is making predictions.

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 74.0 failed 1 times, most recent failure: Lost task 0.0 in stage 74.0 (TID 488, localhost, executor driver): java.lang.ClassCastException: org.apache.spark.ml.linalg.DenseVector cannot be cast to org.apache.spark.mllib.linalg.DenseVector
	at $line148.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:61)
	at $line148.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:61)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:193)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.Task.run(Task.scala:108)
	